# General Assembly DSI - Denver 2018
## Capstone Project - DFS Model
This is my capstone project at General Assembly's fifth [Data Science Immersive](https://generalassemb.ly/education/data-science-immersive) cohort in 2018. I am developing a model to assist in optimizing NFL lineups on the daily fantasy sports platforms [Draft Kings](https://www.draftkings.com/) and [Fan Duel](https://www.fanduel.com/).

### Problem Statement

Can we build a model to predict a football player’s fantasy football performance to estimate their value and implement the model in conjunction with a daily fantasy strategy to be profitable?

In [1]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

%matplotlib inline

/Users/charleydixon/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/charleydixon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
# load data
data = pd.read_csv('../data/modeling_wrs.csv')

In [3]:
data = data.groupby(['Name', 'Year', 'Week', 'Month', 'Team', 'Oppt']).sum()

In [4]:
data.head()

Age  h/a  Favored  Spread  \
Name           Year Week Month     Team Oppt                                 
Adams, Davante 2014 2    September GB   NYJ   21.264    1        1    -7.0   
                    3    September GB   DET   21.271    0        0    -1.5   
                    4    September GB   CHI   21.278    0        1    -2.0   
                    5    October   GB   MIN   21.282    1        1    -8.5   
                    6    October   GB   MIA   21.292    0        1    -1.5   

                                               O/U  Temperature  Wind  \
Name           Year Week Month     Team Oppt                            
Adams, Davante 2014 2    September GB   NYJ   46.5         63.0  11.0   
                    3    September GB   DET   52.0         72.0   0.0   
                    4    September GB   CHI   51.0         78.0   6.0   
                    5    October   GB   MIN   46.5         63.0   7.0   
                    6    October   GB   MIA   47.0         88.0  12.0   

                                              Receptions   Targets   Rec_TDs  \
Name           Year Week Month     Team Oppt                                   
Adams, Davante 2014 2    September GB   NYJ     3.000000  6.333333  0.000000   
                    3    September GB   DET     2.666667  5.000000  0.000000   
                    4    September GB   CHI     3.000000  4.666667  0.000000   
                    5    October   GB   MIN     3.000000  4.333333  0.000000   
                    6    October   GB   MIA     1.666667  3.333333  0.333333   

                                                ...      Weather_DOME  \
Name           Year Week Month     Team Oppt    ...                     
Adams, Davante 2014 2    September GB   NYJ     ...                 0   
                    3    September GB   DET     ...                 1   
                    4    September GB   CHI     ...                 0   
                    5    October   GB   MIN     ...                 0   
                    6    October   GB   MIA     ...                 0   

                                              Weather_Fog  Weather_Rain  \
Name           Year Week Month     Team Oppt                              
Adams, Davante 2014 2    September GB   NYJ             0             0   
                    3    September GB   DET             0             0   
                    4    September GB   CHI             0             0   
                    5    October   GB   MIN             1             0   
                    6    October   GB   MIA             0             0   

                                              Weather_Rain | Fog  \
Name           Year Week Month     Team Oppt                       
Adams, Davante 2014 2    September GB   NYJ                    0   
                    3    September GB   DET                    0   
                    4    September GB   CHI                    0   
                    5    October   GB   MIN                    0   
                    6    October   GB   MIA                    0   

                                              Weather_Snow  \
Name           Year Week Month     Team Oppt                 
Adams, Davante 2014 2    September GB   NYJ              0   
                    3    September GB   DET              0   
                    4    September GB   CHI              0   
                    5    October   GB   MIN              0   
                    6    October   GB   MIA              0   

                                              Weather_Snow | Fog  \
Name           Year Week Month     Team Oppt                       
Adams, Davante 2014 2    September GB   NYJ                    0   
                    3    September GB   DET                    0   
                    4    September GB   CHI                    0   
                    5    October   GB   MIN                    0   
                    6    October   GB   MIA                    0   

  

### Modeling
- Linear Regression
- Random Forest
- Support Vector Regression
- Boosting
- PCA
- Neural Net

### Linear Regression

In [5]:
columns = [col for col in data.columns if col != 'FD points']

def linreg(df, features, target = 'FD points'):
    X = df[features]
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

    # first attempt - all features
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    print('=============EVAULATION=============\n')
    print('R2 Training:', model.score(X_train, y_train))
    print('R2 Testing:', model.score(X_test, y_test))

    predictions = model.predict(X_test)

    print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))
    print('MAE:', mean_absolute_error(y_test, predictions), '\n')
    print('=============COEFFICIENTS=============\n')
    print('Intercept:', model.intercept_)
    for key, index in dict(zip(X.columns, model.coef_)).items():
        print(key, index)
    return model

lr_all_features = linreg(data, columns)

=============EVAULATION=============

R2 Training: 0.21357368801925125
R2 Testing: 0.19696907087177018
RMSE: 6.188420455379233
MAE: 4.8720054200683105 

=============COEFFICIENTS=============

Intercept: -4.885523943097192
Age -0.006861437450359716
h/a 0.2552276368073809
Favored 0.5722819328597738
Spread -0.04549945815720833
O/U 0.04432805984737186
Temperature 0.0035347967703881493
Wind -0.03859521810994177
Receptions 0.08663763483670646
Targets 0.47315325262832025
Rec_TDs -0.310114707651849
Rec_Yds 0.01705604420446762
Opp_Rank -0.03765596806172487
Opp_Avg_Rec_Allowed -0.18453892568859978
Opp_Avg_Targets_Allowed 0.12541076175593396
Opp_Avg_TDs_Allowed 0.1155990766202968
Opp_Avg_Yds_Allowed 0.004851222386516624
Weather_DOME -0.3011350866689547
Weather_Fog 1.1584893507770873
Weather_Rain -1.3974150526353875
Weather_Rain | Fog -1.9376802347148876
Weather_Snow -0.09826526179660736
Weather_Snow | Fog -1.4079746315088875
Weather_Snow | Freezing Rain 4.1168075185894475
Weather_Sunny -0.132826

Feature Coefficients that seem logical:
- Increasing Prediction for:
    - Home/Away
    - Favored
    - O/U
    - Attempts
    - Completions
    - TDs
    - Y/A
    
- Decreasing Prediction for:
    - Age
        - Sort of...
    - Wind
    - Rain
    - Rain & Fog
    - Snow
    - Snow & Fog
    - Interceptions
    - Opponent Rank
        - The rank columns is actually backwards so defenses get better as ranking improves

Feature Coefficients that make no sense to me:
- Increasing Prediction for:
    - Fog
    - Snow & Freezing Rain
    - Opponent Interceptions
- Decreasing Prediction for:
    - Being in a Dome (Controlled conditions seem like a player should perform better)
    - Sunny weather
    - Yards
    - Opponent Attempts Allowed
    - Opponent Completions Allowed
    - Opponent TDs Allowed
- Almost no effect from:
    - Rating
    - FD Salary
        - Considering salary kind of derives FanDuel's predictions I feel like this would have an impact.

**The most interesting thing to note here is that FD Salary has seemingly no effect on a player's point production**

In [6]:
# data.columns

In [7]:
# remove_cols = ['Weather_Fog', 'Weather_Snow | Freezing Rain', 'Opp_Avg_Ints', 'Weather_DOME', 
#                'Weather_Sunny', 'Yards', 'Opp_Avg_Att_Allowed', 'Opp_Avg_Comp_Allowed', 
#                'Opp_Avg_TDs_Allowed', 'Rating', 'FD salary', 'FD points']

# features = [col for col in data.columns if col not in remove_cols]

In [8]:
# lr_partial_features = linreg(data, features)

### Random Forest Regression

In [9]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#### No Hypertuning

In [10]:
model = RandomForestRegressor()
cross_val_score(model, X_train, y_train).mean()

0.0871482416750317

In [11]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [12]:
model.score(X_test, y_test)

0.07771621227128389

In [13]:
predictions = model.predict(X_test)

In [14]:
mean_absolute_error(y_test, predictions)

5.171330900243308

In [15]:
np.sqrt(mean_squared_error(y_test, predictions))

6.6320220870789965

#### With Hypertuning

In [16]:
model = RandomForestRegressor()

params = {
    'max_depth': [10, 20, 30],
    'n_estimators': [70, 75, 80],
    'min_samples_leaf': [13, 14],
    'max_features': [10, 20]
}

gs = GridSearchCV(model, param_grid = params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [10, 20, 30], 'n_estimators': [70, 75, 80], 'min_samples_leaf': [13, 14], 'max_features': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [17]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.2068915814783504
0.2126738536661117
{'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 14, 'n_estimators': 70}


In [18]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

4.803901213315784
6.1276085537387015


In [19]:
dict(zip(X.columns, gs.best_estimator_.feature_importances_))

{'Age': 0.042852465031718405,
 'h/a': 0.0076890290534770206,
 'Favored': 0.006583117747507625,
 'Spread': 0.03186644865114176,
 'O/U': 0.030871149882189933,
 'Temperature': 0.026399446904268903,
 'Wind': 0.021896126138596022,
 'Receptions': 0.1357554470271299,
 'Targets': 0.13454196307860303,
 'Rec_TDs': 0.014690897468737809,
 'Rec_Yds': 0.09176327420856034,
 'Opp_Rank': 0.03405512775163478,
 'Opp_Avg_Rec_Allowed': 0.02722977481038195,
 'Opp_Avg_Targets_Allowed': 0.029434658805580622,
 'Opp_Avg_TDs_Allowed': 0.01274177110292533,
 'Opp_Avg_Yds_Allowed': 0.03440317533452358,
 'Weather_DOME': 0.0017255591316203442,
 'Weather_Fog': 0.0,
 'Weather_Rain': 0.000394286287090537,
 'Weather_Rain | Fog': 0.0,
 'Weather_Snow': 0.0,
 'Weather_Snow | Fog': 0.0,
 'Weather_Snow | Freezing Rain': 0.0,
 'Weather_Sunny': 0.0023892838557111077,
 'FD salary': 0.3127169977286008}

> Now this is telling me that `FD Salary` is far and away the most important feature

### Support Vector Regression

In [67]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#### No Hypertuning

In [21]:
model = SVR()

In [22]:
model.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [23]:
model.score(X_train, y_train)

0.12586005050609317

In [24]:
model.score(X_test, y_test)

-0.0569092544139942

In [25]:
y_pred = model.predict(X_test)

In [26]:
mean_absolute_error(y_test, y_pred)

5.335342506968317

#### Hypertuning

In [68]:
model = SVR()
params = {
    'kernel': ['linear'],
    'C': [0.1, 0.01]
}

gs = GridSearchCV(model, param_grid = params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ['linear'], 'C': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [69]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.18089369199635677
0.19445179950476554
{'C': 0.1, 'kernel': 'linear'}


In [70]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

4.747239064240706
6.198112326554158


### ADA Boost

In [30]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [31]:
model = AdaBoostRegressor()
params = {
    'n_estimators': [45, 47, 50],
    'learning_rate': [.8, .83, .85, .9]
}

gs = GridSearchCV(model, param_grid=params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [45, 47, 50], 'learning_rate': [0.8, 0.83, 0.85, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [32]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.12143584301619473
-0.09575397292913768
{'learning_rate': 0.83, 'n_estimators': 45}


In [33]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

6.137600948984488
7.228866486788187


### Gradient Boost

In [63]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [64]:
model = GradientBoostingRegressor()
params = {
    'n_estimators': [100, 120],
    'learning_rate': [0.15, 0.2, 0.25, 0.3]
}

gs = GridSearchCV(model, param_grid=params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 120], 'learning_rate': [0.15, 0.2, 0.25, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [65]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.18477183788145668
0.20791587163961978
{'learning_rate': 0.15, 'n_estimators': 100}


In [66]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

4.796795583530697
6.1460958963252095


### Principal Component Analysis

In [54]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [55]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

pca = PCA(n_components = 20)
pca.fit(X_train_scaled)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [56]:
var_exp = pca.explained_variance_ratio_
print('Explained Variance: ', var_exp)

Explained Variance:  [0.14938207 0.11103121 0.09854372 0.05612772 0.05153472 0.04414462
 0.04293378 0.04189589 0.0406451  0.04035708 0.03985134 0.03902206
 0.03611149 0.03398105 0.03129266 0.02826511 0.02732568 0.02643202
 0.01888563 0.01484014]


In [57]:
cum_var_exp = np.cumsum(var_exp)
print('Cumulative Explained Variance: ', cum_var_exp)

Cumulative Explained Variance:  [0.14938207 0.26041329 0.35895701 0.41508473 0.46661945 0.51076407
 0.55369785 0.59559373 0.63623883 0.67659591 0.71644725 0.7554693
 0.7915808  0.82556185 0.85685451 0.88511962 0.91244531 0.93887732
 0.95776296 0.97260309]


In [58]:
Z_train = pca.transform(X_train_scaled)
Z_test = pca.transform(X_test_scaled)

In [59]:
model = LinearRegression()

model.fit(Z_train, y_train)
print(model.score(Z_train, y_train))
print(model.score(Z_test, y_test))

0.21128221321554774
0.19741419153531137


In [60]:
predictions = model.predict(Z_test)

In [61]:
mean_absolute_error(y_test, predictions)

4.878558001837594

In [62]:
np.sqrt(mean_squared_error(y_test, predictions))

6.1867050945234805

### Neural Network

In [47]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [48]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [49]:
model = Sequential()

model.add(Dense(X_train.shape[1], activation = 'relu', input_dim = X_train.shape[1]))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(1, activation=None))

In [50]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [51]:
model.fit(X_train_scaled, 
          y_train, 
          epochs = 10, 
          batch_size = 2, 
          validation_data = (X_test_scaled, y_test))

Train on 4930 samples, validate on 1644 samples
Epoch 1/10
4930/4930 [==============================] - 2s 478us/step - loss: 50.9416 - val_loss: 39.1470
Epoch 2/10
4930/4930 [==============================] - 2s 487us/step - loss: 41.9767 - val_loss: 39.2926
Epoch 3/10
4930/4930 [==============================] - 2s 419us/step - loss: 41.2961 - val_loss: 38.6021
Epoch 4/10
4930/4930 [==============================] - 2s 420us/step - loss: 40.9769 - val_loss: 38.2831
Epoch 5/10
4930/4930 [==============================] - 2s 422us/step - loss: 40.6382 - val_loss: 38.2960
Epoch 6/10
4930/4930 [==============================] - 2s 420us/step - loss: 40.3667 - val_loss: 38.8243
Epoch 7/10
4930/4930 [==============================] - 2s 414us/step - loss: 40.0506 - val_loss: 38.3215
Epoch 8/10
4930/4930 [==============================] - 2s 422us/step - loss: 40.1191 - val_loss: 39.9574
Epoch 9/10
4930/4930 [==============================] - 2s 413us/step - loss: 39.8582 - val_loss: 38.828

In [52]:
predictions = model.predict(X_test_scaled)

In [53]:
print(mean_absolute_error(y_test, predictions))
print(np.sqrt(mean_squared_error(y_test, predictions)))

4.809406380911524
6.226977376495484
